https://adventofcode.com/2022/day/24

In [7]:
import numpy as np
import math
from functools import reduce
from heapq import heappush, heappop

In [8]:
with open("data/24.txt") as fh:
    data = fh.read()

In [16]:
testdata = """\
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
"""

In [26]:
def blizzard_builder(data):
    lines = data.strip().splitlines()
    h, w = len(lines) - 2, len(lines[0]) - 2
    north, south, east, west, empty = [np.zeros((h, w), dtype=bool) for _ in range(5)]
    drxnmap = {
        "^": north,
        "v": south,
        ">": east,
        "<": west,
        ".": empty,
    }
    for row, line in enumerate(lines[1:-1]):
        for col, char in enumerate(line[1:-1]):
            drxnmap[char][row, col] = True
    del empty

    header = np.ones((1, w), dtype=bool)
    header[0, 0] = False
    footer = np.ones((1, w), dtype=bool)
    footer[0, -1] = False

    L = []
    for _ in range(math.lcm(h, w)):
        bliz = reduce(np.logical_or, [north, south, east, west])
        L.append(np.vstack([header, bliz, footer]))

        north = np.vstack([north[1:], north[:1]])
        south = np.vstack([south[-1:], south[:-1]])
        east = np.hstack([east[:, -1:], east[:, :-1]])
        west = np.hstack([west[:, 1:], west[:, :1]])

    return L


def open_spots(coords, ar):
    r0, c0 = coords
    L = []
    for (r, c) in [(r0, c0), (r0+1, c0), (r0-1, c0), (r0, c0+1), (r0, c0-1)]:
        if r < 0 or c < 0:
            continue
        try:
            v = ar[r, c]
        except IndexError:
            pass
        else:
            if not v:
                L.append((r, c))
    return L


def dijkstra(data, start=(0, 0), finish=None, initial_state=0):
    states = blizzard_builder(data)
    h, w = states[0].shape
    mod = math.lcm(h-2, w)
    if finish is None:
        finish = (h-1, w-1)
    visited = {(start, initial_state): 0}
    pq = []
    heappush(pq, (0, (start, initial_state)))
    visited_skip_count = 0
    finishers = []
    while pq:
        (d, (p, s)) = heappop(pq)
        if p == finish:
            finishers.append((d, (p, s)))
            continue
        next_s = (s + 1) % mod
        possibles = open_spots(p, states[next_s])
        if not possibles: # don't ever return to this node
            visited[(p, s)] = 0
            continue
        for poss in possibles:
            v = visited.get((poss, next_s))
            if v is None or v > d+1:
                visited[(poss, next_s)] = d+1
                heappush(pq, (d+1, (poss, next_s)))
            else:
                visited_skip_count += 1
    return finishers

In [27]:
%%time
dijkstra(testdata)

CPU times: user 6.54 ms, sys: 394 µs, total: 6.93 ms
Wall time: 5.16 ms


[(18, ((5, 5), 6)),
 (21, ((5, 5), 9)),
 (22, ((5, 5), 10)),
 (23, ((5, 5), 11)),
 (24, ((5, 5), 0)),
 (29, ((5, 5), 5))]

Part 1

In [28]:
%%time
dijkstra(data)

CPU times: user 3.81 s, sys: 117 ms, total: 3.93 s
Wall time: 3.93 s


[(225, ((36, 99), 225)),
 (227, ((36, 99), 227)),
 (229, ((36, 99), 229)),
 (231, ((36, 99), 231)),
 (233, ((36, 99), 233)),
 (234, ((36, 99), 234)),
 (237, ((36, 99), 237)),
 (238, ((36, 99), 238)),
 (242, ((36, 99), 242)),
 (246, ((36, 99), 246)),
 (247, ((36, 99), 247)),
 (250, ((36, 99), 250)),
 (251, ((36, 99), 251)),
 (253, ((36, 99), 253)),
 (254, ((36, 99), 254)),
 (255, ((36, 99), 255)),
 (257, ((36, 99), 257)),
 (258, ((36, 99), 258)),
 (259, ((36, 99), 259)),
 (261, ((36, 99), 261)),
 (270, ((36, 99), 270)),
 (271, ((36, 99), 271)),
 (272, ((36, 99), 272)),
 (273, ((36, 99), 273)),
 (275, ((36, 99), 275)),
 (276, ((36, 99), 276)),
 (278, ((36, 99), 278)),
 (282, ((36, 99), 282)),
 (283, ((36, 99), 283)),
 (284, ((36, 99), 284)),
 (285, ((36, 99), 285)),
 (286, ((36, 99), 286)),
 (287, ((36, 99), 287)),
 (288, ((36, 99), 288)),
 (291, ((36, 99), 291)),
 (292, ((36, 99), 292)),
 (293, ((36, 99), 293)),
 (295, ((36, 99), 295)),
 (296, ((36, 99), 296)),
 (303, ((36, 99), 303)),


Part 2

In [24]:
test_state = blizzard_builder(testdata)[0]
h, w = test_state.shape
mod = math.lcm(h-2, w)
finish = (h-1, w-1)
mod, finish

(12, (5, 5))

In [8]:
first_leg = 18  # from part 1
initial_state = first_leg % mod
dijkstra(testdata, start=finish, finish=(0, 0), initial_state=initial_state)

23

In [9]:
two_legs = 18 + 23
initial_state = two_legs % mod
dijkstra(testdata, start=(0,0), finish=finish, initial_state=initial_state)

13

In [10]:
18 + 23 + 13

54

In [11]:
state = blizzard_builder(data)[0]
h, w = state.shape
mod = math.lcm(h-2, w)
finish = (h-1, w-1)
mod, finish

(700, (36, 99))

In [12]:
first_leg = 225  # from part 1
initial_state = first_leg % mod
dijkstra(data, start=finish, finish=(0, 0), initial_state=initial_state)

238

In [13]:
two_legs = 225 + 238
initial_state = two_legs % mod
dijkstra(data, start=(0,0), finish=finish, initial_state=initial_state)

248

In [14]:
225 + 238 + 248

711